### Ecosystem
Java logging ecosystem consists of `sl4j` and one of many logging frameworks like `log4j`, `logback` or inbuilt Java logging classes. SL4J provides abstraction for various logging frameworks, a bunch of interfaces stating how logging should be done. The logging frameworks provide the actual implementation. The idea is that anyone can switch to his own logging framework of choice by swapping in required jars. Another reason is that library authors want to log the steps being performed by their library but have no idea what logging framework the client maybe using.

Some logging frameworks (like log4j) don't really need sl4j, but it is preferable to use it alongwith sl4j. In this case a sl4j-log4j adaption layer is required. Logback on the other hand is fully based on sl4j. The below diagram explains the  scenario:

![sl4j](images/sl4j.png)

**Basic Usage:** consider the code below:

In [ ]:
import org.slf4j.Logger;
import org.slf4j.LoggerFactory;

public class Sample {
    // or, we can pass a string to name the logger
    // in below case the name of logger will be fully qualified name of class
    final Logger logger = LoggerFactory.getLogger(Sample.class);
    
    // ...

The getLogger method call would look for sl4j bindings in the classpath. If no binding is found on the class path, then slf4j-api will default to a no-operation implementation discarding all log requests.

### Log4J2 
We'll start Log4J2 by writing a small logging code:

In [ ]:
public class Sample {
    static Logger logger = LoggerFactory.getLogger(Sample.class);

    public static void main(String[] args) {
        logger.info("Hello");
    }
}

Nothing gets logged. This is because we didn't specify any logging configuration, so default configuration was used. In fact Log4J2 searches through the following order for logging configuration:
- `log4j.configurationFile` system property, then
- log4j2-test.properties in the classpath, then
- log4j2-test.yaml or log4j2-test.yml in the classpath, then
- log4j2-test.json or log4j2-test.jsn in the classpath, then
- log4j2-test.xml in the classpath, then
- log4j2.properties on the classpath, then
- log4j2.yaml or log4j2.yml in the classpath, then
- log4j2.json or log4j2.jsn in the classpath, then
- log4j2.xml in the classpath, then
- default configuration

In default configuration, only logging level ERROR and above are logged. Other logging levels are:
- FATAL: error messages that indicate that some subsystem or the entire program cannot continue execution and will terminate.
- ERROR: error messages regarding a problem that should be handled by a human. These are generally useful for operators to alert on.
- WARN: warning messages regarding potential problems that may need to be handled by a human. This level is often misused and ignored as a result.
- INFO: informative messages about the state of a program. These types of messages tend to be related to the lifecycle of a program and can be viewed as a way to debug the macro state of the program.
- DEBUG: debugging information about internal states of the program. These messages are usually only helpful to the developers maintaining a program.
- TRACE: messages tracing the execution flow of a program. These messages are usually very low level and simply mirror the micro state of a program and generally don’t offer more information than a debugger would.

Each level can be selectively enabled or disabled, though generally disabling one level will disable all levels below it as well. For example, if we used a logging configuration that was set to WARN as its level, then only WARN, ERROR, and FATAL messages would be shown.

Let us now add a configuration in form of XML file:

```xml
<?xml version="1.0" encoding="UTF-8"?>
<Configuration strict="true">
    <Appenders>
        <Appender type="Console" name="STDOUT">
            <Layout type="PatternLayout" pattern="%d  %-6p  %C{1}.%t:%L %m%n" />
        </Appender>
    </Appenders>
    <Loggers>
        <Root level="trace">
            <AppenderRef ref="STDOUT" />
        </Root>
    </Loggers>
</Configuration>
```

Now our log will be displayed in the console. There are a number of things to discuss from the above configuration. *Appenders* are responsible for delivering logs to their destination. The destination can be console, file, database, etc. Below are some more appender configurations:\

```xml
<Appenders>
    <Appender type="Console" name="STDOUT">
        <Layout type="PatternLayout" pattern="%d  %-6p  %C{1}.%t:%L %m%n" />
    </Appender>
    <Appender type="File" name="FILE" fileName="target/test.log" append="true">
        <Layout type="PatternLayout">
            <Pattern>%d{yyyy-MM-dd HH:mm:ss} %-6p %C{1}.%t:%L >>> %m%n</Pattern>
        </Layout>
    </Appender>
    <Appender type="RollingFile" name="ROLLING"
        fileName="target/rolling.log"
        filePattern="target/rolling-%d{yyyy-MM-dd}-%i.log">
        <Layout type="PatternLayout"
            pattern="%d{yyyy-MM-dd HH:mm:ss}  %-6p  %C{1}.%t:%L  >>> %m%n" />
        <Policies>
            <SizeBasedTriggeringPolicy size="10 KB" />
        </Policies>
        <DefaultRolloverStrategy max="4" />
    </Appender>
</Appenders>
```

If the configuration tag is not set as strict, we can write as:
```xml
<Appenders>
    <File name="FILE" fileName="target/test.log">
      <PatternLayout>
        <Pattern>%d %p %c{1.} [%t] %m%n</Pattern>
      </PatternLayout>
    </File>
    <Console name="CONSOLE" />
    ...
</Appenders>
```

Another tag of interest is *Layout*. An Appender uses a Layout to format a logs into a form that meets the needs of whatever will be consuming the log event. Layout can be CSV, JSON or the most commonly used Pattern Layout.

| Pattern 	| Description 	|
|-	|-	|
| c{precision} 	| Name of the logger that published the logging event. Precision controls size of the logger name. 	|
| C{precision} 	| Fully qualified class name of the caller issuing the logging request. 	|
| d{pattern} 	| Outputs the date of the logging event. Example: `%d{dd MMM yyyy HH:mm:ss}` 	|
| F 	| File name where the logging request was issued. 	|
| L 	| Line number where the logging request was issued. 	|
| m 	| Application supplied message associated with the logging event. 	|
| M 	| Method name where the logging request was issued. 	|
| n 	| Platform dependent line separator character or characters. 	|
| p[{level=label, level=label, ...}] 	| Logging level with optional logging label. Example: `%p{WARN=W, DEBUG=D, ERROR=E, TRACE=T, INFO=I}` 	|
| t 	| Thread name. 	|

The *Logger* tag defines configuration for each Logger. In the above XML config, root logger configuration is defined. This configuration is inherited by all loggers (including the Root Logger). We can also see that we set logging level for the logger in the configuration file. We can also add a particular logger specific configuration as shown below:

```xml
<Root level="warn">
    <AppenderRef ref="STDOUT" />
    <AppenderRef ref="FILE" />
    <AppenderRef ref="ROLLING" />
</Root>
<Logger level="trace" name="com.example.logging">
    <AppenderRef ref="STDOUT" />
</Logger>
```

We get a logger with name com.example.logging.Sample, which inherits from com.example.logging, which inherits from com.example, which inherits from com, which inherits from root.

When we see the log, we would see two log lines (instead of 1 as expected). This is because the appender associated with logger com.example.logging is first used. Next, the parent of com.example.logging, which in this case is the root logger, is referenced. This is known as *additivity*. Since this is undesirable for this case, we set additivity as false

```xml
<Logger level="trace" name="com.example.logging" additivity="false">
    <AppenderRef ref="STDOUT" />
</Logger>
```

Once an event reaches a logger with its additivity set to false the event will not be passed to any of its parent loggers, regardless of their additivity setting.   

In our application, we can get root logger by:

In [ ]:
final Logger rootLogger = LoggerFactory.getLogger(Logger.ROOT_LOGGER_NAME);

String data = "x25GH";
rootLogger.trace("Data : {}, occurance {}", data, 0);